In [1]:
from utils import *

In [2]:
n1 = nn.Sequential(nn.Linear(5,5),nn.ReLU(), nn.Linear(5,1))
n1[0].weight.data = n1[0].weight.data.type(torch.float64)
n1[2].weight.data = n1[2].weight.data.type(torch.float64)
n1[0].bias.data = n1[0].bias.data.type(torch.float64)
n1[2].bias.data = n1[2].bias.data.type(torch.float64)

n1[0].bias.data = torch.zeros_like(n1[0].bias.data)
n1[2].bias.data = torch.zeros_like(n1[2].bias.data)

n1_copy = nn.Sequential(nn.Linear(5,5),nn.ReLU(),nn.Linear(5,1))
copy_weights(n1, n1_copy)
n1_copy[0].weight.data = n1[0].weight.data.type(torch.float64)
n1_copy[2].weight.data = n1[2].weight.data.type(torch.float64)
n1_copy[0].bias.data = n1[0].bias.data.type(torch.float64)
n1_copy[2].bias.data = n1[2].bias.data.type(torch.float64)

n1[0].bias.requires_grad = False
n1[2].bias.requires_grad = False
n1_copy[0].bias.requires_grad = False
n1_copy[2].bias.requires_grad = False

In [3]:
batch_size = 1000
x = 10*torch.randn(batch_size,5, dtype=torch.float64)
targets = 2*torch.randn(batch_size,1, dtype=torch.float64)
criterion = nn.MSELoss()

In [4]:
(n1[0].weight.data - n1_copy[0].weight.data).abs().max()

tensor(0., dtype=torch.float64)

In [5]:
# v = 2*torch.ones_like(v)
v = 1.5*torch.rand(5, dtype=torch.float64) + 0.5
diag_mat = torch.diag(v)
diag_mat_inv = torch.diag(1/v)
n1[0].weight.data = torch.matmul(diag_mat, n1[0].weight.data)
# n1[0].bias.data = torch.matmul(diag_mat, n1[0].bias.data)
n1[2].weight.data = torch.matmul(n1[2].weight.data, diag_mat_inv)

In [6]:
(n1[0].weight.data - n1_copy[0].weight.data).abs().max()

tensor(0.3264, dtype=torch.float64)

In [7]:
diag_mat_sq = torch.matmul(diag_mat, diag_mat)
diag_mat_inv_sq = torch.matmul(diag_mat_inv, diag_mat_inv)

In [8]:
optim1 = torch.optim.SGD(n1.parameters(), lr = 1)
optim2 = torch.optim.SGD(n1_copy.parameters(), lr = 1)

In [9]:
(n1(x) - n1_copy(x)).abs().max()

tensor(3.5527e-15, dtype=torch.float64, grad_fn=<MaxBackward1>)

In [10]:
optim1.zero_grad()
out1 = n1(x)
loss1 = criterion(out1, targets)
loss1.backward()
gradiants_n1 = []
for l in n1:
    if isinstance(l, nn.Linear):
        gradiants_n1.append(l.weight.grad)
        print(l.weight.grad)

tensor([[-0.0442, -0.0722,  0.4110, -0.2565, -0.1130],
        [ 0.6692,  2.4375, -3.3742,  3.8786, -2.5412],
        [ 6.2621, -1.2921,  1.3541,  6.8524,  3.8434],
        [-0.6344, -3.5756,  3.8013, -3.2630,  3.4606],
        [ 2.8798,  2.7469, -3.8654,  5.3020, -2.6298]], dtype=torch.float64)
tensor([[ 3.2670, 28.6084, 16.0039,  1.1901, 11.8808]], dtype=torch.float64)


In [11]:
optim2.zero_grad()
out2 = n1_copy(x)
loss2 = criterion(out2, targets)
loss2.backward()
gradiants_copy = []
for l in n1_copy:
    if isinstance(l, nn.Linear):
        gradiants_copy.append(l.weight.grad)
        print(l.weight.grad)

tensor([[-0.0644, -0.1053,  0.5991, -0.3739, -0.1647],
        [ 1.3332,  4.8562, -6.7222,  7.7272, -5.0627],
        [ 5.2801, -1.0895,  1.1418,  5.7779,  3.2407],
        [-0.4546, -2.5620,  2.7237, -2.3380,  2.4797],
        [ 1.4923,  1.4235, -2.0030,  2.7475, -1.3628]], dtype=torch.float64)
tensor([[ 2.2410, 14.3598, 18.9802,  1.6609, 22.9271]], dtype=torch.float64)


In [12]:
(torch.matmul(diag_mat_inv, gradiants_copy[0]) - gradiants_n1[0]).abs().max()

tensor(3.5527e-15, dtype=torch.float64)

In [13]:
(torch.matmul(gradiants_copy[1], diag_mat) - gradiants_n1[1]).abs().max()

tensor(1.7764e-14, dtype=torch.float64)

In [14]:
# assign gradients
n1_copy[0].weight.grad = torch.matmul(diag_mat_inv_sq, gradiants_copy[0])
n1_copy[2].weight.grad = torch.matmul(gradiants_copy[1], diag_mat_sq)

In [15]:
optim1.step()
optim2.step()

In [16]:
(n1(x) - n1_copy(x)).abs().max()

tensor(5.4570e-12, dtype=torch.float64, grad_fn=<MaxBackward1>)

In [17]:
net = nn.Sequential(nn.Linear(5,5),nn.Linear(5,1))
batch_size = 1000
x = 10*torch.randn(batch_size,5, dtype=torch.float64)
targets = 2*torch.randn(batch_size,1, dtype=torch.float64)
criterion = nn.MSELoss()

In [18]:
v = 1.5*torch.rand(5, dtype=torch.float64) + 0.5
diag_mat = torch.diag(v)
diag_mat_inv = torch.diag(1/v)